In [1]:
import pandas as pd
from sqlalchemy import create_engine
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from pprint import pprint
from sqlalchemy.sql import select
import json
from operator import itemgetter
import numpy as np
import sys

In [2]:
file_one = "static/data/Table.csv"
df = pd.read_csv(file_one, encoding="ISO-8859-1")
df = df.dropna(subset = ['Type of data (a)'])
df = df.rename(columns={"Major area, region, country or area of destination": "Destination"})
df = df.rename(columns={'Unnamed: 6': "Total"})
del df['Sort\norder']
del df['Other South']
del df['Other North']
del df['Notes']
del df['Type of data (a)']
df.index = range(len(df))
df.to_csv("static/data/cleanedData.csv", header=True)
df


,Year,Destination,Code,Total,Afghanistan,Albania,Algeria,American Samoa,Andorra,Angola,...,Uruguay,Uzbekistan,Vanuatu,Venezuela (Bolivarian Republic of),Viet Nam,Wallis and Futuna Islands,Western Sahara,Yemen,Zambia,Zimbabwe
0,1990.0,Burundi,108.0,"333,110",..,..,..,..,..,..,...,..,..,..,..,..,..,..,..,..,..
1,1990.0,Comoros,174.0,"14,079",..,..,..,..,..,..,...,..,..,..,..,..,..,..,..,..,..
2,1990.0,Djibouti,262.0,"122,221",..,..,..,..,..,..,...,..,..,..,..,..,..,..,289,..,..
3,1990.0,Eritrea,232.0,"11,848",..,..,..,..,..,247,...,..,..,..,..,..,..,..,..,91,82
4,1990.0,Ethiopia,231.0,"1,155,390",..,..,..,..,..,..,...,..,..,..,..,..,..,..,68,..,..
5,1990.0,Kenya,404.0,"297,292",..,..,..,..,..,..,...,..,..,..,..,..,..,..,..,..,..
6,1990.0,Madagascar,450.0,"23,917",..,..,..,..,..,..,...,..,..,..,..,..,..,..,..,..,..
7,1990.0,Malawi,454.0,"1,127,724",..,..,..,..,..,..,...,..,..,..,..,..,..,..,..,"5,637","3,851"
8,1990.0,Mauritius,480.0,"3,613",..,..,..,..,..,..,...,..,..,..,..,..,..,..,..,..,..
9,1990.0,Mayotte,175.0,"15,229",..,..,..,..,..,..,...,..,..,..,..,..,..,..,..,..,..


In [3]:
df = pd.read_csv('static/data/cleanedData.csv')
engine = create_engine("sqlite:///static/data/MigrationData.sqlite")


In [4]:
df.head()

,Unnamed: 0,Year,Destination,Code,Total,Afghanistan,Albania,Algeria,American Samoa,Andorra,...,Uruguay,Uzbekistan,Vanuatu,Venezuela (Bolivarian Republic of),Viet Nam,Wallis and Futuna Islands,Western Sahara,Yemen,Zambia,Zimbabwe
0,0,1990.0,Burundi,108.0,"333,110",..,..,..,..,..,...,..,..,..,..,..,..,..,..,..,..
1,1,1990.0,Comoros,174.0,"14,079",..,..,..,..,..,...,..,..,..,..,..,..,..,..,..,..
2,2,1990.0,Djibouti,262.0,"122,221",..,..,..,..,..,...,..,..,..,..,..,..,..,289,..,..
3,3,1990.0,Eritrea,232.0,"11,848",..,..,..,..,..,...,..,..,..,..,..,..,..,..,91,82
4,4,1990.0,Ethiopia,231.0,"1,155,390",..,..,..,..,..,...,..,..,..,..,..,..,..,68,..,..


In [5]:
df = df.replace("..","0")
df.head()

,Unnamed: 0,Year,Destination,Code,Total,Afghanistan,Albania,Algeria,American Samoa,Andorra,...,Uruguay,Uzbekistan,Vanuatu,Venezuela (Bolivarian Republic of),Viet Nam,Wallis and Futuna Islands,Western Sahara,Yemen,Zambia,Zimbabwe
0,0,1990.0,Burundi,108.0,"333,110",0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,1990.0,Comoros,174.0,"14,079",0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,1990.0,Djibouti,262.0,"122,221",0,0,0,0,0,...,0,0,0,0,0,0,0,289,0,0
3,3,1990.0,Eritrea,232.0,"11,848",0,0,0,0,0,...,0,0,0,0,0,0,0,0,91,82
4,4,1990.0,Ethiopia,231.0,"1,155,390",0,0,0,0,0,...,0,0,0,0,0,0,0,68,0,0


In [6]:
sorted_df = df['Total']
sorted_df.dtype
sorted_total = sorted_df.map(lambda x:int(x.replace(",", "")))
sorted_df.head()
df['Total'] = sorted_total
s = df.sort_values(by=['Total'], ascending=False, na_position='last')
# s['Total']

In [7]:
# df['Total'] = pd.to_numeric(df['Total'], errors='coerce')
# df['Total']

In [8]:
data_2017_destination = df.loc[df['Year'] == 2017]
sorted_data_2017 = data_2017_destination.sort_values(by='Total', ascending=False).head(10)
destination_2017 = []
for index, row in sorted_data_2017.iterrows():
    
    dict_2017 = {}
    dict_2017['Destination'] = row['Destination']
    dict_2017['Total'] = row['Total']
    destination_2017.append(dict_2017)
    
destination_2017
# sorted_data_2017
# data_2017_destination

[{'Destination': 'United States of America', 'Total': 49776970},
 {'Destination': 'Saudi Arabia', 'Total': 12185284},
 {'Destination': 'Germany', 'Total': 12165083},
 {'Destination': 'Russian Federation', 'Total': 11651509},
 {'Destination': 'United Kingdom', 'Total': 8841717},
 {'Destination': 'United Arab Emirates', 'Total': 8312524},
 {'Destination': 'France', 'Total': 7902783},
 {'Destination': 'Canada', 'Total': 7861226},
 {'Destination': 'Australia', 'Total': 7035560},
 {'Destination': 'Spain', 'Total': 5947106}]

In [9]:
# cols = []
# for column in data_2017_destination:
#     cols.append(column)
# origins = cols[5:]
# origins
# df1 = data_2017_destination[origins]
# df1['Total'] = pd.to_numeric(df['Total'], errors='coerce')
# origin_list = []
# for origin in origins:
#     origin_dict = {}
#     origin_dict['Origin'] = origin
#     origin_dict['Total'] = df1[origin].sum()
#     origin_list.append(origin_dict)
# origin_list

In [10]:
# Save top 10 Destinations to a JSON file
with open("static/data/top_10_dest.json","w") as f:
    json.dump(destination_2017,f)

In [11]:
engine.table_names()
Base = automap_base()
Base.prepare(engine, reflect=True)
Base.classes.keys()

['mydata']

In [12]:
session = Session(engine) 
Mydata = Base.classes.mydata
conn = engine.connect()

In [13]:
select_data_2017 = select([Mydata]).where(Mydata.Year=='2017')
result = conn.execute(select_data_2017)
# Creating list of dictionaries where year = 2017
resultset = []
for row in result:
    resultset.append(dict(row))
# Removing null values     
newset = []
for res in resultset:
    r = {k: v for k, v in res.items() if v != '..'}
    newset.append(r)
# Creating list of dictionaries with destination, origin countries, ..
data_2017 = []
for res in resultset:
    select_data_2017 = {}
    select_data_2017['destination'] = res.pop('Destination',None)
    select_data_2017['year'] = res.pop('Year',None)
    select_data_2017['code'] = res.pop('Code',None)
    select_data_2017['index'] = res.pop('index',None)
    select_data_2017['origin'] = {k: v for k, v in res.items() if v != '..'}
    data_2017.append(select_data_2017)    

In [14]:
# Finding all country codes
country_code = []
for value in session.query(Mydata.Code).distinct():
     for index, row in df.iterrows():
        if value[0] == row['Code']:
            p = {}
            p['code'] = row['Code']
            p['country'] = row['Destination']
            country_code.append(p)

In [15]:
# Finding unique country codes
unique_country_codes = []

for c in country_code:
    if c not in unique_country_codes:
        if {k: v for k, v in c.items() if v is not None}:
            unique_country_codes.append(c)

In [16]:
# Find country code by name
def getCountryCode(country):
    for c in country_code:
        for k,v in c.items():
            if v == country:
                return c['code']

In [17]:
# Get latitude from country code
def getLatLng(country):
    code = int(getCountryCode(country))
    for l in latlng:
        for k, v in l.items():
            if(k == 'code'):
                if v == code:
                    return [l['longitude'], l['latitude']]


In [18]:
# Create JSON File which countains country code, name , latitude and longitude
df_lat_long = pd.read_csv('static/data/Country_Codes_Latitude_Longitude.csv')
latlng = []
for value in session.query(Mydata.Code).distinct():
     for index, row in df_lat_long.iterrows():
        if value[0] == row['Numeric code']:
            p = {}
            p['code'] = row['Numeric code']
            p['country'] = row['Country']
            p['latitude'] = row['Latitude (average)']
            p['longitude'] = row['Longitude (average)']
            latlng.append(p)
with open("static/data/latlng.json","w") as f:
    json.dump(latlng,f)


In [19]:
# Color code according to number of migrants
def getcolor(migrants):
#     migrants = int(migrants.replace(',', ''))
    if migrants >= 12500000:
        return "#e41a1c"
    elif migrants >= 2500000 and migrants < 12500000:
        return "#377eb8"
    elif migrants >= 1350000 and migrants < 2500000:
        return "#4daf4a"
    elif migrants >= 1000000 and migrants < 1350000:
        return "#984ea3"
    elif migrants >= 750000 and migrants < 1000000:
        return "#ffff33"
    elif migrants >= 600000 and migrants < 750000:
        return "#ff7f00"
    else:
        return "#a65628"


In [20]:
# Create dictionary for using in the plugin
data_for_plugin_2017 = []
for d in data_2017:
    for k, v in d['origin'].items():
        data_dict = {}
        data_dict['to'] = getLatLng(d['destination']) 
        if data_dict['to'] is None:
            data_dict = {}
            continue
        data_dict['from'] = getLatLng(k)
        if data_dict['from'] is None:
            data_dict = {}
            continue
        data_dict['labels1'] = [d['destination'], k]
        v = int(v.replace(',', ''))
        data_dict['color'] = getcolor(v) 
        data_dict['migrants'] = v
        data_dict['arcWidth'] = 0.1
        data_dict['pulseBorderWidth'] = 0.15
        data_dict['year'] = d['year']             
        data_for_plugin_2017.append(data_dict)


In [21]:

list_2 = []

for d in data_for_plugin_2017:
    for k1, v1 in d.items():
            if d['to'] is not None:
                if d['from'] is not None:
                    if d not in list_2:            
                        list_2.append(d)

In [22]:
# Sorted list
sorted_list = sorted(list_2, key=itemgetter('migrants'), reverse=True) 
top_100_list = sorted_list[:100]
top_10_list = sorted_list[:10]
sorted_list_asc = sorted(list_2, key=itemgetter('migrants'), reverse=False)
bottom_10_list = sorted_list_asc[:10]
# top_100_list

In [23]:
# Save top 100 list to a JSON file
with open("static/data/top_100.json","w") as f:
    json.dump(top_100_list,f)

In [24]:
# Save top 10 list to a JSON file
with open("static/data/top_10.json","w") as f:
    json.dump(top_10_list,f)

In [25]:
# Save bottom 10 list to a JSON file
with open("static/data/bottom_10.json","w") as f:
    json.dump(bottom_10_list,f)

In [26]:
# with open("static/data/data_for_plugin_2017.json","w") as f:
#     json.dump(list_2,f)

In [27]:
# Get the name of all columns into a list
# origin_country_list = Mydata.__table__.columns.keys()

# with open('static/data/countries.txt', 'w') as f:
#     for item in origin_country_list:
#         print(item)
